Building the model described in Faizi et. al. 2018 "A model of optimal protein allocation during phototrophic growth" Biosystems, 166: 26-36.

In [6]:
import tellurium as te
import math
import numpy as np
import matplotlib.pyplot as plt

In [7]:
model Faizi
    compartment cell;
    cell = 1.0;
    var Cix in cell; # extracellular inorganic carbon
    var Ci in cell; # intracellular inorganic carbon
    var Energy in cell; # energy units (here representing the ATP, NADH, etc pool)
    var AminoAcid in cell; # amino acids
    #var P0; # inactivated photosynthetic machinery
    #var Pstar; # activated photosynthetic machinery
    var Photoenzyme in cell; # P0+Pstart
    var Transporter in cell; # transporter protein for inorganic carbon
    var Metaenzyme in cell; # metabolic multi-enzyme complex
    var Ribosome in cell; # ribosome
    const Irradiance in cell;
    
    Pm = 0.0018; # permeability, dm min-1
    a_cell = 3.85e-9; # cell surface area, dm2 cell-1
    v_cell = 2.24e-14; # L, cell-1
    N_a = 6.022e23; # Avogadro constant, mol-1
    k_cat_T = 726; # maximal import rate, min-1
    K_T = 15; # half-sat constant of T, uM
    k_cat_M = 545; # maximal metabolic rate, min-1
    K_M = 2441560; # half-sat constant of M, molecules cell-1
    Y_max = 1320; # maximal translation rate, aa min-1 molecules-1
    K_aa = 10000; # half-sat constant for aa, molecule cell-1
    K_e = 10000; # half-sat constant for e, molecule cell-1
    abs_xsec = 0.166; # effective absorption cross-section of P, m2 uE-1
    turnover_P = 1900; # maximal turnover rate of P, min-1
    k_d = 1.6e-6; # rate constant for photodamage
    K_i = 1.0e8; # product-inhibition constant for e synthesis, molecule cell-1
    dens = 1.4e10; # average density, aa cell-1
    n_R = 7358; # ribosome length, aa molecule-1
    n_T = 1681; # transporter length, aa molecule-1
    n_M = 28630; # metabolic enzyme complex length, aa molecule-1
    n_P = 95451; # photosynthetic unit length, aa molecule-1
    m_u = 45; # e needed to create one aa
    m_y = 3; # e needed for one translational elongation step
    m_phi = 8; # e produced by P cycle
    m_c = 5; # average carbon chain length of aa
    
    beta_R = 0.25;
    beta_T = 0.25;
    beta_P = 0.25;
    beta_M = 0.25;
    
       
    Cix=1000;
    Ci=0;
    Energy=K_e;
    AminoAcid=K_aa;
    Metaenzyme=1;
    Ribosome=1;
    Transporter=1;
    Photoenyzme=1;
    Irradiance=1000;
    
    
    v_t := Transporter * k_cat_T * (Cix/(K_T+Cix)) * (Energy/(K_e+Energy));
    v_m := Metaenzyme * k_cat_M * (Ci/(K_M+Ci)) * (Energy/(K_e+Energy));
    gamma_R := beta_R * (Y_max / n_R) * Ribosome * (Energy/(K_e+Energy)) * (AminoAcid/(K_aa+AminoAcid));
    gamma_T := beta_T * (Y_max / n_T) * Ribosome * (Energy/(K_e+Energy)) * (AminoAcid/(K_aa+AminoAcid));
    gamma_M := beta_M * (Y_max / n_M) * Ribosome * (Energy/(K_e+Energy)) * (AminoAcid/(K_aa+AminoAcid));
    gamma_P := beta_P * (Y_max / n_P) * Ribosome * (Energy/(K_e+Energy)) * (AminoAcid/(K_aa+AminoAcid));
    k2e := turnover_P / (1+(Energy/K_i)^4);
    v2 := (k2e*abs_xsec*Irradiance*Photoenzyme)/(abs_xsec*Irradiance+k2e+mu);
    translation := (n_R*gamma_R + n_T*gamma_T + n_M*gamma_M + n_P*gamma_P);
    mu := (1/dens) * (v_t/m_c);
    
    AA_P: => AminoAcid; v_m;
    AA_C: AminoAcid => ; translation + mu*AminoAcid;
    
    CI_P: => Ci; v_t;
    CI_C: Ci => ; v_m*m_c + mu*Ci;
    
    M_P: => Metaenzyme; gamma_M;
    M_C: Metaenzyme =>; mu*Metaenzyme;
    
    T_P: => Transporter; gamma_T;
    T_C: Transporter => ; mu*Transporter;
    
    R_P: => Ribosome; gamma_R;
    R_C: Ribosome => ; mu*Ribosome;
    
    P_P: => Photoenzyme; gamma_P;
    P_C: Photoenzyme => ; mu*Photoenzyme;
    
    e_P: => Energy; m_phi*v2;
    e_C: Energy => ; v_t+v_m*m_u + mu*Energy + m_y*translation;
    
    
end

Success: Model can be accessed via variable Faizi


In [9]:
Faizi.reset()
Faizi.simulate(start = 0, end = 5.64e14, steps = 1000,selections =['time','Ci','Energy','AminoAcid'])
Faizi.plot()

Faizi.steadyStateSelections = ['Ci','Energy','AminoAcid']
values = Faizi.getSteadyStateValues()
selections = Faizi.steadyStateSelections
print(values)
print(selections)

[  5.43425576e+10   9.64646774e+01   1.18701299e+03]
['Ci', 'Energy', 'AminoAcid']
